# Prepare DoS Hulk & Goldeneye Closed Port Attack Dataset
* 'Closed Port' means that the DoS attack is performed on an HTTP server that is <u>not</u> currently running.

## Overview:

This notebook will focus on creating a DoS Hulk & Goldeneye closed port attack dataset based on a small sample of data collected by performing real DoS HTTP GET Flood attacks on a webserver that is offline in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 3,000 flows of the DoS HTTP GET Flood closed port attack based on the samples we collected when running a DoS HTTP GET Flood attacks in various configurations using the well known DoS Hulk and DoS Goldeneye tools when the victim web server was offline.<br> 
The victim web server was a regular Flask web server.

## Imports & Global Variables:

In [9]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 3000
ATTACK_NAME = 'DoS'

In [10]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the sample dataset:

In [11]:
# import the attack sample dataset
dos_samples = pd.read_csv('dos_hulk_goldeneye_samples_closed_port.csv')
print(f'Dataset Shape: {dos_samples.shape}')
dos_samples

Dataset Shape: (8, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,74.0,74,74,0.0,0.0,20240,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,3373.333333,506,0,0,1.260528,401.419060,0.814980,0.002496,0.036519
1,1,74.0,74,74,0.0,0.0,20280,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,2897.142857,507,0,0,2.241037,226.234538,0.956779,0.004429,0.056727
2,1,74.0,74,74,0.0,0.0,20560,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1468.571429,514,0,0,3.208441,160.202415,1.023966,0.006254,0.069266
3,1,74.0,74,74,0.0,0.0,20480,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1706.666667,512,0,0,2.302322,222.384193,0.895812,0.004506,0.054606
4,1,74.0,74,74,0.0,0.0,36680,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,58.407643,917,0,0,18.027555,50.866576,8.869790,0.019681,0.319971
5,1,74.0,74,74,0.0,0.0,24640,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,77.728707,616,0,0,7.291802,84.478433,3.481875,0.011857,0.144467
6,1,74.0,74,74,0.0,0.0,24360,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,78.834951,609,0,0,9.749890,62.462243,6.866742,0.016036,0.278521
7,1,74.0,74,74,0.0,0.0,12320,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1540.000000,308,0,0,2.122772,145.093304,0.767681,0.006915,0.045732


### Find the columns that we need to synthesis data for:

In [12]:
columns_to_gather = dos_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist()  #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [13]:
# find the minimum and maximum values for each column, scale the range (reduce min by 10% and increase max by 10%), and store the results in a dictionary.
min_max_dict = {col: (float(dos_samples[col].min() * 0.9), float(dos_samples[col].max() * 1.1)) for col in columns_to_gather}
min_max_dict['Number of Ports'] = (1, 1)

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 1.00 | Max: 1.00
Average Packet Length          | Min: 66.60 | Max: 81.40
Packet Length Min              | Min: 66.60 | Max: 81.40
Packet Length Max              | Min: 66.60 | Max: 81.40
Total Length of Fwd Packet     | Min: 11088.00 | Max: 40348.00
Fwd Packet Length Max          | Min: 36.00 | Max: 44.00
Fwd Packet Length Mean         | Min: 36.00 | Max: 44.00
Fwd Packet Length Min          | Min: 36.00 | Max: 44.00
Subflow Fwd Bytes              | Min: 52.57 | Max: 3710.67
SYN Flag Count                 | Min: 277.20 | Max: 1008.70
Flow Duration                  | Min: 1.13 | Max: 19.83
Packets Per Second             | Min: 45.78 | Max: 441.56
IAT Max                        | Min: 0.69 | Max: 9.76
IAT Mean                       | Min: 0.00 | Max: 0.02
IAT Std                        | Min: 0.03 | Max: 0.35


### Create the base attack dataset (full of zeros):

In [14]:
# creating an empty dataframe before adding values to it
dos_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dos_samples.columns))), columns = dos_samples.columns)
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [15]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dos_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dos_dataset[col] = int(0)
zero_columns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

---

## Filling in values based on collected samples:

### Firstly we insert data into columns that have the exact same values:

In [16]:
same_value = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max']
val = np.random.randint(min_max_dict[same_value[0]][0], min_max_dict[same_value[0]][1]*1.1, NUM_OF_ROWS)

for col in same_value:
    dos_dataset[col] = val

In [17]:
same_value2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']
val2 = np.random.randint(min_max_dict[same_value2[0]][0], min_max_dict[same_value2[0]][1]*1.25, NUM_OF_ROWS)

for col in same_value2:
    dos_dataset[col] = val2

### Then we insert data into columns that are independant of each other, based on the min max values:

In [18]:
dos_dataset['Flow Duration'] = np.random.uniform(min_max_dict['Flow Duration'][0]*0.95, min_max_dict['Flow Duration'][1]*1.05, NUM_OF_ROWS)
dos_dataset['Number of Ports'] = np.full(shape = NUM_OF_ROWS, fill_value = 1, dtype = int)
dos_dataset['Subflow Fwd Bytes'] = np.full(shape = NUM_OF_ROWS, fill_value = 0, dtype = int)
dos_dataset['SYN Flag Count'] = np.random.randint(min_max_dict['SYN Flag Count'][0]*0.9, min_max_dict['SYN Flag Count'][1]*1.1, NUM_OF_ROWS)

## Then we fill values into columns that have a certain correlation between them:

### Correlation between 'SYN Flag Count' and 'Total Length of Fwd Packet':

In [ ]:
# finding the correlation between the 'SYN Flag Count' column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'Total Length of Fwd Packet']
independent_col = dos_samples[first_correlation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dos_samples[first_correlation[1]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'SYN Flag Count' and 'Total Length of Fwd Packet'
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)
    
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'SYN Flag Count'
        delta = random.uniform(factor * 0.05, factor * 0.25) 
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = int(row['SYN Flag Count'] * updated_factor)

('Total Length of Fwd Packet', np.float64(40.00000000000001))


### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std':

In [ ]:
# finding the correlation between the 'Flow Duration' column to the rest of the columns in order to create new data
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dos_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Flow Duration'
dependent_cols = dos_samples[second_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'Flow Duration' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(dos_samples['Flow Duration'].values, dos_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

('Packets Per Second', np.float64(8.979156689418554))
('IAT Max', np.float64(0.5248448565577644))
('IAT Mean', np.float64(0.001311749361677395))
('IAT Std', np.float64(0.020313533395492172))


np.float64(561.125)

In [21]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Flow Duration'
        if col == 'Packets Per Second':
            delta = random.uniform(duration_to_packets_corr*0.05, duration_to_packets_corr * 0.125) 
            updated_factor = duration_to_packets_corr + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = updated_factor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.1, factor * 0.25)
                updated_factor = factor + random.choices([-1, 1], weights=[1, 2], k=1)[0] * delta  
            elif col == 'IAT Max':
                delta = random.uniform(factor * 0.1, factor * 0.25)
                updated_factor = factor + random.choice([-1, 1]) * delta  
            else:
                delta = random.uniform(factor * 0.25, factor * 0.5)
                updated_factor = factor + random.choices([-1, 1], weights=[1, 3], k=1)[0] * delta
            dos_dataset.loc[index, col] = row['Flow Duration'] * updated_factor

In our sample dataset, the column 'Subflow Fwd Bytes' usually has values in a specific range, but sometimes it has values outside of the range.<br>
In order to generate accurate data, we generate a vector that will have a certain distribution of values. For example, in the 'Subflow Fwd Bytes' column, 75% of the values will be within the usual range,<br>
but the other 25% will have values that are anywhere between the minimal and maximal value for this column, meaning they will have values outside of the usual range as well.  

In [22]:
rand_values = np.random.uniform(min_max_dict['Subflow Fwd Bytes'][0], min_max_dict['Subflow Fwd Bytes'][1], NUM_OF_ROWS)
usual_values = np.random.uniform(1105.763, 3642.146, NUM_OF_ROWS)

# Choose values randomly (25% from rand_values, 75% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.25, usual_values, rand_values) 

dos_dataset['Subflow Fwd Bytes'] = chosen_values

---

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [23]:
dos_samples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,74.0,74,74,0.0,0.0,20240,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,3373.333333,506,0,0,1.260528,401.419060,0.814980,0.002496,0.036519
1,1,74.0,74,74,0.0,0.0,20280,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,2897.142857,507,0,0,2.241037,226.234538,0.956779,0.004429,0.056727
2,1,74.0,74,74,0.0,0.0,20560,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1468.571429,514,0,0,3.208441,160.202415,1.023966,0.006254,0.069266
3,1,74.0,74,74,0.0,0.0,20480,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1706.666667,512,0,0,2.302322,222.384193,0.895812,0.004506,0.054606
4,1,74.0,74,74,0.0,0.0,36680,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,58.407643,917,0,0,18.027555,50.866576,8.869790,0.019681,0.319971
5,1,74.0,74,74,0.0,0.0,24640,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,77.728707,616,0,0,7.291802,84.478433,3.481875,0.011857,0.144467
6,1,74.0,74,74,0.0,0.0,24360,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,78.834951,609,0,0,9.749890,62.462243,6.866742,0.016036,0.278521
7,1,74.0,74,74,0.0,0.0,12320,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1540.000000,308,0,0,2.122772,145.093304,0.767681,0.006915,0.045732


In [24]:
dos_samples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,8.0,8.0,8.0,8.0,8.0,8.0,8.000000,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.000000,8.000000,8.0,8.0,8.000000,8.000000,8.000000,8.000000,8.000000
mean,1.0,74.0,74.0,74.0,0.0,0.0,22445.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400.085698,561.125000,0.0,0.0,5.775543,169.142595,2.959703,0.009022,0.125726
std,0.0,0.0,0.0,0.0,0.0,0.0,6872.302588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1284.388466,171.807565,0.0,0.0,5.770372,115.414002,3.204683,0.006176,0.112626
min,1.0,74.0,74.0,74.0,0.0,0.0,12320.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.407643,308.000000,0.0,0.0,1.260528,50.866576,0.767681,0.002496,0.036519
25%,1.0,74.0,74.0,74.0,0.0,0.0,20270.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.558390,506.750000,0.0,0.0,2.211471,78.974385,0.875604,0.004486,0.052388
50%,1.0,74.0,74.0,74.0,0.0,0.0,20520.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1504.285714,513.000000,0.0,0.0,2.755381,152.647860,0.990372,0.006584,0.062997
75%,1.0,74.0,74.0,74.0,0.0,0.0,24430.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2004.285714,610.750000,0.0,0.0,7.906324,223.346779,4.328092,0.012901,0.177980
max,1.0,74.0,74.0,74.0,0.0,0.0,36680.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3373.333333,917.000000,0.0,0.0,18.027555,401.419060,8.869790,0.019681,0.319971


In [25]:
dos_dataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,3000.0,3000.000000,3000.000000,3000.000000,3000.0,3000.0,3000.000000,3000.000000,3000.000000,3000.000000,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.000000,3000.000000,3000.0,3000.0,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,1.0,77.199000,77.199000,77.199000,0.0,0.0,27041.780333,44.935667,44.935667,44.935667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2249.391394,674.548000,0.0,0.0,11.118614,83.090328,5.862240,0.017265,0.237786
std,0.0,6.620524,6.620524,6.620524,0.0,0.0,10897.557789,5.515705,5.515705,5.515705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,853.902132,248.530622,0.0,0.0,5.773385,82.312959,3.275755,0.010532,0.130950
min,1.0,66.000000,66.000000,66.000000,0.0,0.0,7665.000000,36.000000,36.000000,36.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.737076,249.000000,0.0,0.0,1.089069,23.805804,0.471979,0.000814,0.017506
25%,1.0,72.000000,72.000000,72.000000,0.0,0.0,18024.250000,40.000000,40.000000,40.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1570.836592,455.750000,0.0,0.0,5.998517,34.986828,3.048591,0.008291,0.125436
50%,1.0,77.000000,77.000000,77.000000,0.0,0.0,26421.000000,45.000000,45.000000,45.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2260.541462,674.500000,0.0,0.0,11.269091,50.220064,5.719808,0.015288,0.233135
75%,1.0,83.000000,83.000000,83.000000,0.0,0.0,34600.750000,50.000000,50.000000,50.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2960.407420,890.000000,0.0,0.0,16.107686,94.582215,8.284999,0.026111,0.338739
max,1.0,88.000000,88.000000,88.000000,0.0,0.0,54948.000000,54.000000,54.000000,54.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3704.393044,1108.000000,0.0,0.0,20.796718,570.953163,13.595461,0.040192,0.526421


---

## Adding the Label column:

In [26]:
dos_dataset['Label'] = ATTACK_NAME

---

## At the end we save the dataset as a CSV file

In [27]:
print(f'Attack Dataset Shape: {dos_dataset.shape}')

Attack Dataset Shape: (3000, 27)


In [ ]:
# save the dataset
dos_dataset.to_csv('dos_hulk_goldeneye_closed_port_dataset.csv', index=False)